<div style="text-align: center; font-family: 'charter bt pro roman'; color: rgb(0, 65, 75);">
    <h1>
    GDP Vintages, Releases and Revisions datasets
    </h1>
</div>

<div style="text-align: center; font-family: 'charter bt pro roman'; color: rgb(0, 65, 75);">
<h3>
Documentation
<br>
____________________
<br>
</h3>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    This 
    <span style="color: rgb(0, 65, 75);">jupyter notebook</span>
    provides a step-by-step guide to <b>data building</b> regarding the project <b>'Revisiones y sesgos en las estimaciones preliminares del PBI en el Perú'</b>. This guide covers the creation of GDP mid-term revision dataset for each sector. A key step is the construction at par of what we will call “The ‘t+h’ structure”. This dataset is similar to that of the GDP growth vintages by sector, but instead of growth rate values, it contains values of type “t+h”, where h indicates how many months have passed since the preliminary growth rate was first published; that is, this jupyter notebook also covers the creation of vintages datasets of growth rates associated with a horizon (<b>h</b>).
</div>

<div style="text-align: center; font-family: 'PT Serif Pro Book'; color: rgb(0, 65, 75); font-size: 16px;">
    Jason Cruz
    <br>
    <a href="mailto:jj.cruza@up.edu.pe" style="color: rgb(0, 153, 123); font-size: 16px;">
        jj.cruza@up.edu.pe
    </a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;line-height: 1.5;">
<span style="font-size: 34px;">&#128452;</span> The 't+h' structure should be available for all sectors and frequencies.
    <br>
    <span style="font-size: 24px;">&#8987;</span> Available since <b>1994-2024</b> (Table 1) and since <b>1997-2024</b> (Table 2). 
    <br>
</div>

<div style="font-family: Amaya; text-align: left; color: rgb(0, 65, 75); font-size:16px">The following <b>outline is functional</b>. By utilising the provided buttons, users are able to enhance their experience by browsing this script.<div/>

<div id="outilne">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #292929; padding: 10px; line-height: 1.5; font-family: 'PT Serif Pro Book';">
    <h2 style="text-align: left; color: #E0E0E0;">
        Outline
    </h2>
    <br>
    <a href="#libraries" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        Libraries</a>
    <br>
    <a href="#setup" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        Initial set-up</a>
    <br>
    <a href="#1" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        1. Economic sector selector</a>
    <br>
    <a href="#2" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        2. Create horizon datasets</a>
    <br>
    <a href="#2.1." style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        2.1. Loading growth rate datasets from postgresql.</a>
    <br>
    <a href="#2.2." style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        2.2. Creating horizon dataset step by step.</a> 
    <br>
    <a href="#3" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        3. Create base year datasets</a>
    <br>
    <a href="#3.1." style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        3.1. Loading growth rate datasets from postgresql.</a>
    <br>
    <a href="#3.2." style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        3.2. Creating base year dataset.</a> 
    <br>
    <a href="#4" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        4. Remove observations affected by base year</a>
    <br>
    <a href="#5" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        5. Create growth rates by horizon dataset</a>
    <br>
    <a href="#6" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        6. Create vintages and releases datasets</a>
    <br>
    <a href="#7" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        7. Loading to SQL</a>
    <br>
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Any questions or issues regarding the coding, please email Jason Cruz <a href="mailto:jj.cruza@alum.up.edu.pe" style="color: rgb(0, 153, 123); text-decoration: none;"><span style="font-size: 24px;">&#x2709;</span>
    </a>.
    <div/>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    If you don't have the libraries below, please use the following code (as example) to install the required libraries.
    <div/>

In [ ]:
#!pip install os # Comment this code with "#" if you have already installed this library.

<div id="libraries">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark;">
    <h2>
    Libraries
    </h2>
    <div/>

In [1]:
# POSTGRESSQL
import os
from sqlalchemy import create_engine

# HORIZON DATASETS
import pandas as pd
import numpy as np
import re


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="setup">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark;">
    <h2>
    Initial set-up
    </h2>
    <div/>

<p style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> The following function will establish a connection to the <code>gdp_revisions_datasets</code> database in <code>PostgreSQL</code>. The <b>input data</b> used in this jupyter notebook will be loaded from this <code>PostgreSQL</code> database, and similarly, all <b>output data</b> generated by this jupyter notebook will be stored in that database. Ensure that you set the necessary parameters to access the server once you have obtained the required permissions.<p/>
    
<p style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
To request permissions, please email Jason Cruz <a href="mailto:jj.cruza@alum.up.edu.pe" style="color: rgb(0, 153, 123); text-decoration: none;"> <span style="font-size: 24px;">&#x2709;</span>
    </a>.
<p/>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    <span style="font-size: 24px; color: #FFA823; font-weight: bold;">&#9888;</span>
    Enter your user credentials to acces to SQL.
    <div/>

In [2]:
def create_sqlalchemy_engine():
    """
    Function to create an SQLAlchemy engine using environment variables.
    
    Returns:
        engine: SQLAlchemy engine object.
    """
    # Get environment variables
    user = os.environ.get('CIUP_SQL_USER')  # Get the SQL user from environment variables
    password = os.environ.get('CIUP_SQL_PASS')  # Get the SQL password from environment variables
    host = os.environ.get('CIUP_SQL_HOST')  # Get the SQL host from environment variables
    port = 5432  # Set the SQL port to 5432
    database = 'gdp_revisions_datasets'  # Set the database name 'gdp_revisions_datasets' from SQL

    # Check if all environment variables are defined
    if not all([host, user, password]):
        raise ValueError("Some environment variables are missing (CIUP_SQL_HOST, CIUP_SQL_USER, CIUP_SQL_PASS)")

    # Create connection string
    connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

    # Create SQLAlchemy engine
    engine = create_engine(connection_string)
    
    return engine

<div style="text-align: left;">
    <span style="font-size: 24px; color: rgb(255, 32, 78); font-weight: bold;">&#9888;</span>
    <span style="font-family: PT Serif Pro Book; color: black; font-size: 16px;">
        Import all other functions required by this jupyter notebook.
    </span>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Please, check the script <code>gdp_revisions_datasets_functions.py</code> which contains all the functions required by this jupyter notebook. The functions there are ordered according to the <a href="#outilne" style="color: #3d30a2;">sections</a> of this jupyter notebok.<div/>

In [3]:
from gdp_revisions_datasets_functions import *

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="1">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book; color: dark;">1.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Economic sector and data frequency selector</span></h1>

<div id="steps-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#step-1-1" style="text-decoration: none; color: #006769"> <span style="font-size: 24px; color: rgb(0, 65, 75);">&#10122;</span> Select economic sector</a>
    <br>
    <a href="#step-1-2" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> Select frequency</a>
</div>

<div id="step-1-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10122;</span> <span>Select economic sector</span>
  </div>

In [472]:
# Call the function to show the popup window
sector = show_option_window()
print("Selected economic sector:", sector)

Selected economic sector: electricity


<div id="step-1-2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> <span>Select frequency</span>
  </div>

In [473]:
# Call the function to show the popup window
frequency = show_frequency_window()
print("Selected frequency:", frequency)

Selected frequency: quarterly


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-1" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-1" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="2">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book; color: dark;">2.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Create horizon datasets</span></h1>

<div id="2.1.">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">2.1. </span> <span style = "color: dark; font-family: PT Serif Pro Book;">Loading growth rate datasets from <code>PostgresSQL</code></span></h2>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Connect to SQL.
    <div/>

In [474]:
# Connect to SQL
engine = create_sqlalchemy_engine()

# SQL Query
query = f"SELECT * FROM {sector}_{frequency}_growth_rates;" # Please change your query to PosgtresSQL as you see fit

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Comment the code below if you want the default option (display rows and columns of the dataframe in a limited way)
    <div/>

In [475]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Check the loaded dataframe.
    <div/>

In [476]:
# Read growth rates dataset as DataFrame
globals()[f'{sector}_{frequency}_growth_rates'] = pd.read_sql(query, engine)
growth_rates = globals()[f'{sector}_{frequency}_growth_rates']
growth_rates_df = growth_rates.copy()
growth_rates_df.head(10)

,year,id_ns,date,1994_1,1994_2,1994_3,1994_4,1995_1,1995_2,1995_3,1995_4,1996_1,1996_2,1996_3,1996_4,1997_1,1997_2,1997_3,1997_4,1998_1,1998_2,1998_3,1998_4,1999_1,1999_2,1999_3,1999_4,2000_1,2000_2,2000_3,2000_4,2001_1,2001_2,2001_3,2001_4,2002_1,2002_2,2002_3,2002_4,2002_2.1,2003_1,2003_2,2003_3,2003_4,2004_1,2004_2,2004_3,2004_4,2005_1,2005_2,2005_3,2005_4,2006_1,2006_2,2006_3,2006_4,2007_1,2007_2,2007_3,2007_4,2008_1,2008_2,2008_3,2008_4,2009_1,2009_2,2009_3,2009_4,2010_1,2010_2,2010_3,2010_4,2011_1,2011_2,2011_3,2011_4,2012_1,2012_2,2012_3,2012_4,2013_1,2013_2,2013_3,2013_4,2014_1,2014_2,2014_3,2014_4,2015_1,2015_2,2015_3,2015_4,2016_1,2016_2,2016_3,2016_4,2017_1,2017_2,2017_3,2017_4,2018_1,2018_2,2018_3,2018_4,2019_1,2019_2,2019_3,2019_4,2020_1,2020_2,2020_3,2020_4,2021_1,2021_2,2021_3,2021_4,2022_1,2022_2,2022_3,2022_4,2023_1,2023_2,2023_3,2023_4,2024_1
0,2003,19,2003-05-23,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2.3,None,4.1,5.2,6.1,3.8,4.0,5.0,6.3,5.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003,20,2003-05-30,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2.3,None,4.1,5.2,6.1,3.8,4.0,5.0,6.3,5.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2003,21,2003-06-06,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2.3,None,4.1,5.2,6.1,3.8,4.0,5.0,6.3,5.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2003,22,2003-06-13,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2.3,None,4.1,5.2,6.1,3.8,4.0,5.0,6.3,5.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2003,23,2003-06-20,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2.3,None,4.1,5.2,6.1,3.8,4.0,5.0,6.3,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2003,24,2003-06-27,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2.3,None,4.1,5.2,6.1,3.8,4.0,5.0,6.3,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [443]:
# Find duplicates of id_ns within the same year
duplicated_rows = growth_rates_df[growth_rates_df.duplicated(subset=['year', 'id_ns'], keep=False)]
duplicated_rows

,year,id_ns,date,1994_1,1994_2,1994_3,1994_4,1995_1,1995_2,1995_3,1995_4,1996_1,1996_2,1996_3,1996_4,1997_1,1997_2,1997_3,1997_4,1998_1,1998_2,1998_3,1998_4,1999_1,1999_2,1999_3,1999_4,2000_1,2000_2,2000_3,2000_4,2001_1,2001_2,2001_3,2001_4,2002_1,2002_2,2002_3,2002_4,2002_2.1,2003_1,2003_2,2003_3,2003_4,2004_1,2004_2,2004_3,2004_4,2005_1,2005_2,2005_3,2005_4,2006_1,2006_2,2006_3,2006_4,2007_1,2007_2,2007_3,2007_4,2008_1,2008_2,2008_3,2008_4,2009_1,2009_2,2009_3,2009_4,2010_1,2010_2,2010_3,2010_4,2011_1,2011_2,2011_3,2011_4,2012_1,2012_2,2012_3,2012_4,2013_1,2013_2,2013_3,2013_4,2014_1,2014_2,2014_3,2014_4,2015_1,2015_2,2015_3,2015_4,2016_1,2016_2,2016_3,2016_4,2017_1,2017_2,2017_3,2017_4,2018_1,2018_2,2018_3,2018_4,2019_1,2019_2,2019_3,2019_4,2020_1,2020_2,2020_3,2020_4,2021_1,2021_2,2021_3,2021_4,2022_1,2022_2,2022_3,2022_4,2023_1,2023_2,2023_3,2023_4,2024_1


In [444]:
growth_rates_df.iloc[:10,:5]

,year,id_ns,date,1994_1,1994_2
0,1997,01,1997-01-10,11.3,16.2
1,1997,02,1997-01-17,11.3,16.2
2,1997,03,1997-01-24,11.3,16.2
3,1997,04,1997-01-31,11.3,16.2
4,1997,05,1997-02-07,11.3,16.2
5,1997,06,1997-02-14,11.3,16.2
6,1997,07,1997-02-21,11.3,16.2
7,1997,08,1997-02-28,11.3,16.2
8,1997,09,1997-03-07,11.3,16.2
9,1997,10,1997-03-14,11.3,16.2


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="2.2.">
   <!-- Contenido de la celda de destino -->
</div>

<div id="2.3.">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">2.2.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    Creating horizon dataset step by step
    </span>
    </h2>

<div id="steps-2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#step-2-1" style="text-decoration: none; color: #006769"> <span style="font-size: 24px; color: rgb(0, 65, 75);">&#10122;</span> Replace decimal values by “t+h” values only in the rows representing a new rung</a>
    <br>
    <a href="#step-2-2" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> Concatenate first 3 columns: year, date, id_ns </a>
    <br>
    <a href="#step-2-3" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> Convert columns to string type</a>
    <br>
    <a href="#step-2-4" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> Spreads the "t+h" values over the remaining decimal values </a>
    <br>
    <a href="#step-2-5" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> Exporting to excel file </a>
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
   <span>Set key variables to fill the growth_rates with 't+h' values</span>
  </div>

In [445]:
# Call the function to show the h_initial window
h_initial = show_h_initial_window()
print("Selected h_initial:", h_initial)

Selected h_initial: 3


In [446]:
# Call the function to show the start_row window
start_row = show_start_row_window()
print("Selected start_row:", start_row)

Selected start_row: 0


In [447]:
# Define the mapping of frequencies to h_counter values
frequency_mapping = {
    'monthly': 1,
    'quarterly': 3,
    'annual': 12
}

# Get the appropriate h_counter value based on the selected frequency
h_counter = frequency_mapping.get(frequency)
print("Selected h_counter:", h_counter)

Selected h_counter: 3


<div id="step-2-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10122;</span> <span>Replace decimal values by “t+h” values only in the rows representing a new rung</span>
  </div>

In [448]:
horizon = replace_horizon(growth_rates_df.iloc[:, 3:], start_row, h_initial, h_counter)
horizon_df = horizon.copy()
horizon_df.head(10)

,1994_1,1994_2,1994_3,1994_4,1995_1,1995_2,1995_3,1995_4,1996_1,1996_2,1996_3,1996_4,1997_1,1997_2,1997_3,1997_4,1998_1,1998_2,1998_3,1998_4,1999_1,1999_2,1999_3,1999_4,2000_1,2000_2,2000_3,2000_4,2001_1,2001_2,2001_3,2001_4,2002_1,2002_2,2002_3,2002_4,2002_2.1,2003_1,2003_2,2003_3,2003_4,2004_1,2004_2,2004_3,2004_4,2005_1,2005_2,2005_3,2005_4,2006_1,2006_2,2006_3,2006_4,2007_1,2007_2,2007_3,2007_4,2008_1,2008_2,2008_3,2008_4,2009_1,2009_2,2009_3,2009_4,2010_1,2010_2,2010_3,2010_4,2011_1,2011_2,2011_3,2011_4,2012_1,2012_2,2012_3,2012_4,2013_1,2013_2,2013_3,2013_4,2014_1,2014_2,2014_3,2014_4,2015_1,2015_2,2015_3,2015_4,2016_1,2016_2,2016_3,2016_4,2017_1,2017_2,2017_3,2017_4,2018_1,2018_2,2018_3,2018_4,2019_1,2019_2,2019_3,2019_4,2020_1,2020_2,2020_3,2020_4,2021_1,2021_2,2021_3,2021_4,2022_1,2022_2,2022_3,2022_4,2023_1,2023_2,2023_3,2023_4,2024_1
0,t+33,t+30,t+27,t+24,t+21,t+18,t+15,t+12,t+9,t+6,t+3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11.3,16.2,16.3,19.0,10.8,5.9,3.7,-1.5,0.9,1.8,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11.3,16.2,16.3,19.0,10.8,5.9,3.7,-1.5,0.9,1.8,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11.3,16.2,16.3,19.0,10.8,5.9,3.7,-1.5,0.9,1.8,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11.3,16.2,16.3,19.0,10.8,5.9,3.7,-1.5,0.9,1.8,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,t+34,t+31,t+28,t+25,t+22,t+19,t+16,t+13,t+10,t+7,t+4,t+1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,11.3,16.2,16.3,19.0,10.8,5.9,3.7,-1.5,0.9,1.8,2.5,5.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

<div id="step-2-2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> <span>Concatenate first 3 columns: year, date, id_ns</span>
  </div>

In [449]:
# Get the first three columns of the original DataFrame
first_3_columns = growth_rates_df.iloc[:, :3]

# Concatenate the first three columns with h_{sector}_{frequency}_growth_rates
horizon_df = pd.concat([first_3_columns, horizon_df], axis=1)
horizon_df.head(20)

,year,id_ns,date,1994_1,1994_2,1994_3,1994_4,1995_1,1995_2,1995_3,1995_4,1996_1,1996_2,1996_3,1996_4,1997_1,1997_2,1997_3,1997_4,1998_1,1998_2,1998_3,1998_4,1999_1,1999_2,1999_3,1999_4,2000_1,2000_2,2000_3,2000_4,2001_1,2001_2,2001_3,2001_4,2002_1,2002_2,2002_3,2002_4,2002_2.1,2003_1,2003_2,2003_3,2003_4,2004_1,2004_2,2004_3,2004_4,2005_1,2005_2,2005_3,2005_4,2006_1,2006_2,2006_3,2006_4,2007_1,2007_2,2007_3,2007_4,2008_1,2008_2,2008_3,2008_4,2009_1,2009_2,2009_3,2009_4,2010_1,2010_2,2010_3,2010_4,2011_1,2011_2,2011_3,2011_4,2012_1,2012_2,2012_3,2012_4,2013_1,2013_2,2013_3,2013_4,2014_1,2014_2,2014_3,2014_4,2015_1,2015_2,2015_3,2015_4,2016_1,2016_2,2016_3,2016_4,2017_1,2017_2,2017_3,2017_4,2018_1,2018_2,2018_3,2018_4,2019_1,2019_2,2019_3,2019_4,2020_1,2020_2,2020_3,2020_4,2021_1,2021_2,2021_3,2021_4,2022_1,2022_2,2022_3,2022_4,2023_1,2023_2,2023_3,2023_4,2024_1
0,1997,01,1997-01-10,t+33,t+30,t+27,t+24,t+21,t+18,t+15,t+12,t+9,t+6,t+3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1997,02,1997-01-17,11.3,16.2,16.3,19.0,10.8,5.9,3.7,-1.5,0.9,1.8,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1997,03,1997-01-24,11.3,16.2,16.3,19.0,10.8,5.9,3.7,-1.5,0.9,1.8,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1997,04,1997-01-31,11.3,16.2,16.3,19.0,10.8,5.9,3.7,-1.5,0.9,1.8,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1997,05,1997-02-07,11.3,16.2,16.3,19.0,10.8,5.9,3.7,-1.5,0.9,1.8,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1997,06,1997-02-14,t+34,t+31,t+28,t+25,t+22,t+19,t+16,t+13,t+10,t+7,t+4,t+1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1997,07,1997-02-21,11.3,16.2,1

<div id="step-2-3">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> <span>Convert columns to string type</span>
  </div>

In [450]:
horizon_df = columns_str(horizon_df)
horizon_df.head(20)

,year,id_ns,date,1994_1,1994_2,1994_3,1994_4,1995_1,1995_2,1995_3,1995_4,1996_1,1996_2,1996_3,1996_4,1997_1,1997_2,1997_3,1997_4,1998_1,1998_2,1998_3,1998_4,1999_1,1999_2,1999_3,1999_4,2000_1,2000_2,2000_3,2000_4,2001_1,2001_2,2001_3,2001_4,2002_1,2002_2,2002_3,2002_4,2002_2.1,2003_1,2003_2,2003_3,2003_4,2004_1,2004_2,2004_3,2004_4,2005_1,2005_2,2005_3,2005_4,2006_1,2006_2,2006_3,2006_4,2007_1,2007_2,2007_3,2007_4,2008_1,2008_2,2008_3,2008_4,2009_1,2009_2,2009_3,2009_4,2010_1,2010_2,2010_3,2010_4,2011_1,2011_2,2011_3,2011_4,2012_1,2012_2,2012_3,2012_4,2013_1,2013_2,2013_3,2013_4,2014_1,2014_2,2014_3,2014_4,2015_1,2015_2,2015_3,2015_4,2016_1,2016_2,2016_3,2016_4,2017_1,2017_2,2017_3,2017_4,2018_1,2018_2,2018_3,2018_4,2019_1,2019_2,2019_3,2019_4,2020_1,2020_2,2020_3,2020_4,2021_1,2021_2,2021_3,2021_4,2022_1,2022_2,2022_3,2022_4,2023_1,2023_2,2023_3,2023_4,2024_1
0,1997,01,1997-01-10,t+33,t+30,t+27,t+24,t+21,t+18,t+15,t+12,t+9,t+6,t+3,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1997,02,1997-01-17,11.3,16.2,16.3,19.0,10.8,5.9,3.7,-1.5,0.9,1.8,2.5,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,1997,03,1997-01-24,11.3,16.2,16.3,19.0,10.8,5.9,3.7,-1.5,0.9,1.8,2.5,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,1997,04,1997-01-31,11.3,16.2,16.3,19.0,10.8,5.9,3.7,-1.5,0.9,1.8,2.5,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,1997,05,1997-02-07,11.3,16.2,16.3,19.0,10.8,5.9,3.7,-1.5,0.9,1.8,2.5,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,1997,06,1997-02-14,t+34,t+31,t+28,t+25,t+22,t+19,t+16,t+13,t+10,t+7,t+4,t+1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,1997,07,1997-02-21,11.3,16.2,16.3,19.0,10.8,5.9,3.7,-1.5,0.9,1.8,2.5,5.3,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7,1997,08,1997-02-28,11.3,16.2,16.3,19.0,10.8,5.9,3.7,-1.5,0.9,1.8,2.5,5.3,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8,1997,09,1997-03-07,11.3,16.2,16.3,19.0,10.8,5.9,3.7,-1.5,0.9,1.8,2.5,5.3,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9,1997,10,1997-03-14,11.3,16.2,16.3,19.0,10.8,5.9,3.7,-1.5,0.9,1.8,2.5,5.3,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


<div id="step-2-4">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> Spreads the "t+h" values over the remaining decimal values
  </div>

In [451]:
horizon_df = replace_horizon_1(horizon_df)
horizon_df.head(20)

,year,id_ns,date,1994_1,1994_2,1994_3,1994_4,1995_1,1995_2,1995_3,1995_4,1996_1,1996_2,1996_3,1996_4,1997_1,1997_2,1997_3,1997_4,1998_1,1998_2,1998_3,1998_4,1999_1,1999_2,1999_3,1999_4,2000_1,2000_2,2000_3,2000_4,2001_1,2001_2,2001_3,2001_4,2002_1,2002_2,2002_3,2002_4,2002_2.1,2003_1,2003_2,2003_3,2003_4,2004_1,2004_2,2004_3,2004_4,2005_1,2005_2,2005_3,2005_4,2006_1,2006_2,2006_3,2006_4,2007_1,2007_2,2007_3,2007_4,2008_1,2008_2,2008_3,2008_4,2009_1,2009_2,2009_3,2009_4,2010_1,2010_2,2010_3,2010_4,2011_1,2011_2,2011_3,2011_4,2012_1,2012_2,2012_3,2012_4,2013_1,2013_2,2013_3,2013_4,2014_1,2014_2,2014_3,2014_4,2015_1,2015_2,2015_3,2015_4,2016_1,2016_2,2016_3,2016_4,2017_1,2017_2,2017_3,2017_4,2018_1,2018_2,2018_3,2018_4,2019_1,2019_2,2019_3,2019_4,2020_1,2020_2,2020_3,2020_4,2021_1,2021_2,2021_3,2021_4,2022_1,2022_2,2022_3,2022_4,2023_1,2023_2,2023_3,2023_4,2024_1
0,1997,01,1997-01-10,t+33,t+30,t+27,t+24,t+21,t+18,t+15,t+12,t+9,t+6,t+3,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1997,02,1997-01-17,t+33,t+30,t+27,t+24,t+21,t+18,t+15,t+12,t+9,t+6,t+3,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,1997,03,1997-01-24,t+33,t+30,t+27,t+24,t+21,t+18,t+15,t+12,t+9,t+6,t+3,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,1997,04,1997-01-31,t+33,t+30,t+27,t+24,t+21,t+18,t+15,t+12,t+9,t+6,t+3,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,1997,05,1997-02-07,t+34,t+31,t+28,t+25,t+22,t+19,t+16,t+13,t+10,t+7,t+4,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,1997,06,1997-02-14,t+34,t+31,t+28,t+25,t+22,t+19,t+16,t+13,t+10,t+7,t+4,t+1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,1997,07,1997-02-21,t+34,t+31,t+28,t+25,t+22,t+19,t+16,t+13,t+10,t+7,t+4,t+1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7,1997,08,1997-02-28,t+34,t+31,t+28,t+25,t+22,t+19,t+16,t+13,t+10,t+7,t+4,t+1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8,1997,09,1997-03-07,t+35,t+32,t+29,t+26,t+23,t+20,t+17,t+14,t+11,t+8,t+5,t+2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9,1997,10,1997-03-14,t+35,t+32,t+29,t+26,t+23,t+20,t+17,t+14,t+11,t+8,t+5,t+2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


<div id="step-2-5">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> Exporting to excel file
  </div>

In [452]:
#def export_to_excel(df, filename):
#    # Exportar el DataFrame como un archivo Excel
#    df.to_excel(filename, index=False)

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-2" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-2" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="3">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Create base year datasets</span></h1>

<div id="3.1.">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.1. </span> <span style = "color: dark; font-family: PT Serif Pro Book;">Loading growth rate datasets from <code>PostgresSQL</code></span></h2>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Connect to SQL.
    <div/>

In [453]:
# SQL Query
query = f"SELECT * FROM ns_base_year;" # Please change your query to PosgtresSQL as you see fit

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Check the loaded dataframe.
    <div/>

In [454]:
# Read growth rates dataset as DataFrame
ns_base_year = pd.read_sql(query, engine)
ns_base_year_df = ns_base_year.copy()
ns_base_year_df.head(10)

,year,id_ns,date,base_year
0,1994,01,1994-01-10,1990
1,1994,02,1994-01-17,1990
2,1994,03,1994-01-24,1990
3,1994,04,1994-02-01,1990
4,1994,05,1994-02-07,1990
5,1994,06,1994-02-14,1990
6,1994,07,1994-02-21,1990
7,1994,08,1994-02-28,1990
8,1994,09,1994-03-04,1990
9,1994,10,1994-03-14,1990


<div id="3.2.">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.2.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    Creating base year dataset
    </span>
    </h2>

In [455]:
base_year_df = replace_floats_with_base_year(ns_base_year_df, growth_rates_df)

In [456]:
base_year_df.head(10)

,year,id_ns,date,1994_1,1994_2,1994_3,1994_4,1995_1,1995_2,1995_3,1995_4,1996_1,1996_2,1996_3,1996_4,1997_1,1997_2,1997_3,1997_4,1998_1,1998_2,1998_3,1998_4,1999_1,1999_2,1999_3,1999_4,2000_1,2000_2,2000_3,2000_4,2001_1,2001_2,2001_3,2001_4,2002_1,2002_2,2002_3,2002_4,2002_2.1,2003_1,2003_2,2003_3,2003_4,2004_1,2004_2,2004_3,2004_4,2005_1,2005_2,2005_3,2005_4,2006_1,2006_2,2006_3,2006_4,2007_1,2007_2,2007_3,2007_4,2008_1,2008_2,2008_3,2008_4,2009_1,2009_2,2009_3,2009_4,2010_1,2010_2,2010_3,2010_4,2011_1,2011_2,2011_3,2011_4,2012_1,2012_2,2012_3,2012_4,2013_1,2013_2,2013_3,2013_4,2014_1,2014_2,2014_3,2014_4,2015_1,2015_2,2015_3,2015_4,2016_1,2016_2,2016_3,2016_4,2017_1,2017_2,2017_3,2017_4,2018_1,2018_2,2018_3,2018_4,2019_1,2019_2,2019_3,2019_4,2020_1,2020_2,2020_3,2020_4,2021_1,2021_2,2021_3,2021_4,2022_1,2022_2,2022_3,2022_4,2023_1,2023_2,2023_3,2023_4,2024_1
0,1997,01,1997-01-10,1990,1990,1990,1990,1990,1990,1990,1990,1990,1990,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1997,02,1997-01-17,1990,1990,1990,1990,1990,1990,1990,1990,1990,1990,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1997,03,1997-01-24,1990,1990,1990,1990,1990,1990,1990,1990,1990,1990,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1997,04,1997-01-31,1990,1990,1990,1990,1990,1990,1990,1990,1990,1990,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1997,05,1997-02-07,1990,1990,1990,1990,1990,1990,1990,1990,1990,1990,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1997,06,1997-02-14,1990,1990,1990,1990,1990,1990,1990,1990,1990,1990,1990,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1997

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Consider the code below as provisional. Actually, no data should be exported to the current directory folder, all data should be uploaded to SQL.
    <div/>

In [457]:
# Export to excel file
#with pd.ExcelWriter('base_year_df.xlsx') as writer:
#    base_year_df.to_excel(writer, sheet_name='base_year_df', index=False) # this is an optional, to view data generated

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="4">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">4.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Remove observations affected by base year</span></h1>

<div id="steps-4">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#st-4-1" style="text-decoration: none; color: #006769"> <span style="font-size: 24px; color: rgb(0, 65, 75);">&#10122;</span> Generating a dictionary to match observations affected by base year</a>
    <br>
    <a href="#st-4-2" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> Remove observations affected by base year</a>
    <br>
</div>

<div id="st-4-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10122;</span> <span>Generating a dictionary to match observations affected by base year</span>
  </div>

In [458]:
base_year_dictionary = create_dic_base_year(base_year_df)

In [459]:
base_year_dictionary

{'1998_1': {173,
  174,
  175,
  176,
  177,
  178,
  179,
  180,
  181,
  182,
  183,
  184,
  185,
  186,
  187,
  188,
  189,
  190,
  191,
  192,
  193,
  194,
  195,
  196,
  197,
  198,
  199,
  200,
  201,
  202,
  203,
  204,
  205,
  206,
  207,
  208,
  209,
  210,
  211,
  212,
  213},
 '1998_2': {173,
  174,
  175,
  176,
  177,
  178,
  179,
  180,
  181,
  182,
  183,
  184,
  185,
  186,
  187,
  188,
  189,
  190,
  191,
  192,
  193,
  194,
  195,
  196,
  197,
  198,
  199,
  200,
  201,
  202,
  203,
  204,
  205,
  206,
  207,
  208,
  209,
  210,
  211,
  212,
  213},
 '1998_3': {173,
  174,
  175,
  176,
  177,
  178,
  179,
  180,
  181,
  182,
  183,
  184,
  185,
  186,
  187,
  188,
  189,
  190,
  191,
  192,
  193,
  194,
  195,
  196,
  197,
  198,
  199,
  200,
  201,
  202,
  203,
  204,
  205,
  206,
  207,
  208,
  209,
  210,
  211,
  212,
  213},
 '1998_4': {173,
  174,
  175,
  176,
  177,
  178,
  179,
  180,
  181,
  182,
  183,
  184,
  185,
  186

<div id="st-4-2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> <span>Remove observations affected by base year</span>
  </div>

In [460]:
input_growth_rates_df = remove_base_year_affected_obs(base_year_dictionary, growth_rates_df)

In [461]:
input_growth_rates_df.head(30)

,year,id_ns,date,1994_1,1994_2,1994_3,1994_4,1995_1,1995_2,1995_3,1995_4,1996_1,1996_2,1996_3,1996_4,1997_1,1997_2,1997_3,1997_4,1998_1,1998_2,1998_3,1998_4,1999_1,1999_2,1999_3,1999_4,2000_1,2000_2,2000_3,2000_4,2001_1,2001_2,2001_3,2001_4,2002_1,2002_2,2002_3,2002_4,2002_2.1,2003_1,2003_2,2003_3,2003_4,2004_1,2004_2,2004_3,2004_4,2005_1,2005_2,2005_3,2005_4,2006_1,2006_2,2006_3,2006_4,2007_1,2007_2,2007_3,2007_4,2008_1,2008_2,2008_3,2008_4,2009_1,2009_2,2009_3,2009_4,2010_1,2010_2,2010_3,2010_4,2011_1,2011_2,2011_3,2011_4,2012_1,2012_2,2012_3,2012_4,2013_1,2013_2,2013_3,2013_4,2014_1,2014_2,2014_3,2014_4,2015_1,2015_2,2015_3,2015_4,2016_1,2016_2,2016_3,2016_4,2017_1,2017_2,2017_3,2017_4,2018_1,2018_2,2018_3,2018_4,2019_1,2019_2,2019_3,2019_4,2020_1,2020_2,2020_3,2020_4,2021_1,2021_2,2021_3,2021_4,2022_1,2022_2,2022_3,2022_4,2023_1,2023_2,2023_3,2023_4,2024_1
0,1997,01,1997-01-10,11.3,16.2,16.3,19.0,10.8,5.9,3.7,-1.5,0.9,1.8,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1997,02,1997-01-17,11.3,16.2,16.3,19.0,10.8,5.9,3.7,-1.5,0.9,1.8,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1997,03,1997-01-24,11.3,16.2,16.3,19.0,10.8,5.9,3.7,-1.5,0.9,1.8,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1997,04,1997-01-31,11.3,16.2,16.3,19.0,10.8,5.9,3.7,-1.5,0.9,1.8,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1997,05,1997-02-07,11.3,16.2,16.3,19.0,10.8,5.9,3.7,-1.5,0.9,1.8,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1997,06,1997-02-14,11.3,16.2,16.3,19.0,10.8,5.9,3.7,-1.5,0.9,1.8,2.5,5.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1997,07,1997-02-21,11.3,16.2,16.3,1

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Consider the code below as provisional. Actually, no data should be exported to the current directory folder, all data should be uploaded to SQL.
    <div/>

In [462]:
# Export to excel file
with pd.ExcelWriter('input_growth_rates_df.xlsx') as writer:
    input_growth_rates_df.to_excel(writer, sheet_name='input_growth_rates_df', index=False) # this is an optional, to view data generated

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="5">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">5.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Create growth rates by horizon dataset</span></h1>

<div id="steps-5">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#st-3-1" style="text-decoration: none; color: #006769"> <span style="font-size: 24px; color: rgb(0, 65, 75);">&#10122;</span> Generating a dictionary with the row indices and their t+h values</a>
    <br>
    <a href="#st-3-2" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> Colapse growth rates by horizon ('t+h') </a>
    <br>
</div>

<div id="st-5-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10122;</span> <span>Generating a dictionary with the row indices and their t+h values</span>
  </div>

In [463]:
horizon_dictionary = get_last_index_h(horizon_df)

<div id="st-5-2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> <span>Colapse growth rates by horizon ('t+h')  </span>
  </div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Note that <code>'t'</code> is the date of the first announcement, while <code>'t+h'</code> is 'h' months after the first announcement.
    <div/>

In [464]:
filtered_h_df = filter_df_by_indices(input_growth_rates_df, horizon_dictionary)
filtered_h_df

,horizon,1994_1,1994_2,1994_3,1994_4,1995_1,1995_2,1995_3,1995_4,1996_1,1996_2,1996_3,1996_4,1997_1,1997_2,1997_3,1997_4,1998_1,1998_2,1998_3,1998_4,1999_1,1999_2,1999_3,1999_4,2000_1,2000_2,2000_3,2000_4,2001_1,2001_2,2001_3,2001_4,2002_1,2002_2,2002_3,2002_4,2002_2.1,2003_1,2003_2,2003_3,2003_4,2004_1,2004_2,2004_3,2004_4,2005_1,2005_2,2005_3,2005_4,2006_1,2006_2,2006_3,2006_4,2007_1,2007_2,2007_3,2007_4,2008_1,2008_2,2008_3,2008_4,2009_1,2009_2,2009_3,2009_4,2010_1,2010_2,2010_3,2010_4,2011_1,2011_2,2011_3,2011_4,2012_1,2012_2,2012_3,2012_4,2013_1,2013_2,2013_3,2013_4,2014_1,2014_2,2014_3,2014_4,2015_1,2015_2,2015_3,2015_4,2016_1,2016_2,2016_3,2016_4,2017_1,2017_2,2017_3,2017_4,2018_1,2018_2,2018_3,2018_4,2019_1,2019_2,2019_3,2019_4,2020_1,2020_2,2020_3,2020_4,2021_1,2021_2,2021_3,2021_4,2022_1,2022_2,2022_3,2022_4,2023_1,2023_2,2023_3,2023_4,2024_1
0,t+1,11.3,16.2,16.3,19.0,10.8,5.9,3.7,-1.5,0.9,1.8,2.5,5.3,3.0,12.4,8.1,2.9,1.7,-9.1,2.3,-5.4,2.0,8.0,-0.2,20.7,20.6,9.5,6.4,-1.2,-3.4,-0.1,-2.0,-0.9,-0.7,4.3,5.6,7.3,4.3,4.6,0.7,1.7,-0.2,5.0,6.2,6.1,9.3,4.9,7.3,6.0,5.3,8.0,3.7,8.8,9.1,9.2,12.2,10.4,10.7,10.0,9.8,7.4,3.3,-5.1,-11.1,-10.8,-2.3,7.3,17.0,18.0,13.0,12.0,5.6,3.7,1.0,-0.9,0.1,3.5,2.2,-0.2,3.1,1.4,1.3,3.5,-3.3,-3.7,-10.0,-5.1,-0.1,-2.4,0.9,-3.0,-7.9,2.0,2.2,1.7,3.5,-3.0,-5.4,0.5,10.7,1.9,10.3,-1.0,-6.9,4.0,-2.3,-9.6,-34.0,-9.9,1.0,15.6,60.9,8.4,1.3,1.9,3.5,0.4,-2.3,-0.3,-11.7,-9.0,-4.8,-6.3
1,t+2,11.3,16.2,16.3,19.0,10.8,5.9,3.7,-1.5,0.9,1.8,2.5,5.3,3.0,12.4,8.1,2.9,0.4,-10.7,1.3,-4.1,2.3,6.9,0.1,20.7,20.6,9.5,6.4,-1.2,-3.4,-0.1,-2.0,-0.9,-0.7,4.3,5.6,7.3,4.3,4.6,0.7,1.7,-0.2,5.0,6.2,6.1,9.3,4.9,7.3,6.0,5.3,6.3,2.8,8.0,8.2,8.4,10.8,10.1,10.7,9.9,9.8,7.4,3.3,-5.1,-11.1,-10.8,-2.3,7.3,17.0,18.0,13.0,12.0,5.6,3.7,1.0,-0.9,0.1,3.5,2.2,-0.2,3.1,1.4,NaN,2.9,-3.4,-3.1,-10.0,-5.2,-0.2,-2.3,0.9,-3.0,-7.9,2.0,2.2,1.7,3.5,-3.0,-5.4,1.0,10.8,1.6,11.4,-0.9,-6.8,3.1,-2.3,-9.6,-34.0,-9.9,1.0,15.6,60.9,8.4,1.3,1.9,3.5,0.4,-2.3,-0.3,-11.7,-9.0,-4.8,-6.3
2,t+3,11.3,16.2,16.3,19.0,10.8,5.9,3.7,-1.5,0.9,1.8,2.5,5.3,3.0,12.4,8.1,2.9,0.4,-10.7,1.3,-4.1,2.3,6.9,0.1,20.7,NaN,9.5,6.4,-1.2,-3.4,-0.1,-2.0,-0.9,-0.7,4.3,5.6,7.3,4.3,5.4,0.7,1.7,-0.2,5.0,6.2,6.1,9.3,4.9,7.3,6.0,5.3,6.5,2.8,8.0,8.2,8.4,10.8,10.1,10.7,9.9,9.8,7.4,3.3,-5.1,-11.1,-10.8,-2.3,7.3,17.0,18.0,13.0,12.0,5.6,3.7,1.0,-0.9,0.1,3.5,2.2,-0.2,3.1,1.4,NaN,2.9,-3.4,-3.1,-10.0,-5.2,-0.2,-2.3,0.9,-3.0,-7.9,2.0,2.2,1.7,3.5,-3.0,-5.4,1.0,10.8,1.6,11.4,-0.9,-6.8,3.1,-2.3,-9.6,-34.0,-9.9,1.0,15.6,60.9,8.4,1.3,1.9,3.5,0.4,-2.3,-0.3,-11.7,-9.0,-5.7,NaN
3,t+4,11.3,16.2,16.3,19.0,10.8,5.9,3.7,-1.5,0.9,1.8,2.5,5.0,3.0,12.4,8.1,3.1,0.4,-10.7,1.3,-4.1,2.3,6.9,0.1,20.7,NaN,9.1,6.4,-1.2,-3.6,-0.1,-0.8,-0.9,-0.7,4.3,5.6,7.3,4.3,5.5,0.8,1.8,-0.2,4.7,6.1,6.6,9.1,5.6,7.3,7.0,5.2,6.3,2.8,8.0,8.2,8.4,10.8,10.1,10.7,10.1,10.0,9.9,3.4,-5.3,-11.7,-10.3,-2.3,7.5,17.4,17.4,13.0,12.1,5.8,3.8,1.0,-0.8,0.2,3.6,2.2,-0.3,3.2,1.7,NaN,2.9,-3.4,-3.1,-10.0,-5.2,-0.2,-2.3,0.9,-2.7,-8.2,2.2,2.6,1.9,3.8,-1.7,-5.3,1.0,10.8,1.6,11.4,-0.9,-6.8,3.1,-2.3,-9.6,-34.4,-8.0,1.2,16.7,60.9,8.3,2.7,1.9,3.6,1.1,-2.4,-0.4,-11.7,-9.0,-5.2,NaN
4,t+5,11.7,17.2,16.2,19.3,10.4,5.0,3.8,-1.5,1.1,2.3,2.4,5.0,3.0,12.4,8.1,3.1,0.9,-10.7,2.0,-4.1,2.3,7.8,-0.2,20.7,NaN,9.1,6.4,-1.2,-3.1,-0.1,-0.8,-0.9,-0.7,4.3,5.6,7.3,4.3,5.5,0.8,1.8,-0.2,4.7,6.1,6.6,9.1,5.6,7.3,7.0,4.3,6.8,2.9,8.1,8.2,8.7,11.4,10.4,11.8,10.1,10.0,9.9,3.4,-5.3,-11.7,-10.3,-2.3,7.5,17.4,17.4,13.0,12.1,5.8,3.8,1.0,-0.8,0.2,3.6,2.2,-0.3,3.2,NaN,NaN,4.4,-3.2,-3.4,-10.0,-5.0,-0.2,-2.4,0.9,-2.7,-8.2,2.2,2.6,1.9,3.8,-1.7,-5.3,0.5,10.9,1.7,11.4,-0.9,-6.7,4.0,-2.3,-9.6,-34.4,-8.0,1.2,16.7,60.9,8.3,2.7,1.9,3.6,1.1,-2.4,-0.4,-11.7,-9.0,-5.2,NaN
5,t+6,NaN,NaN,NaN,NaN,10.4,5.0,3.8,-1.5,1.1,2.3,2.4,5.0,3.0,12.4,8.1,3.1,0.9,-10.7,2.0,-5.4,2.3,7.8,-0.2,NaN,NaN,9.1,6.4,-1.6,-3.1,-0.1,-0.8,-0.9,-0.7,4.3,5.6,7.3,4.3,5.5,0.8,1.8,-0.2,4.7,6.1,6.6,9.1,5.6,7.3,7.0,4.5,6.8,2.9,8.1,8.2,8.7,11.4,10.4,11.8,10.1,10.0,9.9,3.4,-5.3,-11.7,-10.3,-2.3,7.5,17.4,17.4,13.0,12.1,5.8,3.8,1.0,-0.8,0.2,3.6,2.2,-0.3,3

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-5" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-5" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="6">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">6.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Create vintages and releases datasets</span></h1>

<div id="steps-6">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#st-4-1" style="text-decoration: none; color: #006769"> <span style="font-size: 24px; color: rgb(0, 65, 75);">&#10122;</span> Create vintages</a>
    <br>
    <a href="#st-4-2" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> Clean-up vintages dataframe </a>
    <br>
    <a href="#st-4-3" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> Create releases </a>
    <br>
</div>

<div id="st-6-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10122;</span> <span>Create vintages</span>
  </div>

In [465]:
vintages = create_vintages(input_growth_rates_df)
vintages.head(30)

aux,vintages_date,ns_1997m01,ns_1997m02,ns_1997m03,ns_1997m04,ns_1997m05,ns_1997m06,ns_1997m07,ns_1997m08,ns_1997m09,ns_1997m10,ns_1997m11,ns_1997m12,ns_1998m01,ns_1998m02,ns_1998m03,ns_1998m04,ns_1998m05,ns_1998m06,ns_1998m07,ns_1998m08,ns_1998m09,ns_1998m10,ns_1998m11,ns_1998m12,ns_1999m01,ns_1999m02,ns_1999m03,ns_1999m04,ns_1999m05,ns_1999m06,ns_1999m07,ns_1999m08,ns_1999m09,ns_1999m10,ns_1999m11,ns_1999m12,ns_2000m01,ns_2000m02,ns_2000m03,ns_2000m04,ns_2000m05,ns_2000m06,ns_2000m07,ns_2000m08,ns_2000m09,ns_2000m10,ns_2000m11,ns_2000m12,ns_2001m01,ns_2001m02,ns_2001m03,ns_2001m04,ns_2001m05,ns_2001m06,ns_2001m07,ns_2001m08,ns_2001m09,ns_2001m10,ns_2001m11,ns_2001m12,ns_2002m01,ns_2002m02,ns_2002m03,ns_2002m04,ns_2002m05,ns_2002m06,ns_2002m07,ns_2002m08,ns_2002m09,ns_2002m10,ns_2002m11,ns_2002m12,ns_2003m01,ns_2003m02,ns_2003m03,ns_2003m04,ns_2003m05,ns_2003m06,ns_2003m07,ns_2003m08,ns_2003m09,ns_2003m10,ns_2003m11,ns_2003m12,ns_2004m01,ns_2004m02,ns_2004m03,ns_2004m04,ns_2004m05,ns_2004m06,ns_2004m07,ns_2004m08,ns_2004m09,ns_2004m10,ns_2004m11,ns_2004m12,ns_2005m01,ns_2005m02,ns_2005m03,ns_2005m04,ns_2005m05,ns_2005m06,ns_2005m07,ns_2005m08,ns_2005m09,ns_2005m10,ns_2005m11,ns_2005m12,ns_2006m01,ns_2006m02,ns_2006m03,ns_2006m04,ns_2006m05,ns_2006m06,ns_2006m07,ns_2006m08,ns_2006m09,ns_2006m10,ns_2006m11,ns_2006m12,ns_2007m01,ns_2007m02,ns_2007m03,ns_2007m04,ns_2007m05,ns_2007m06,ns_2007m07,ns_2007m08,ns_2007m09,ns_2007m10,ns_2007m11,ns_2007m12,ns_2008m01,ns_2008m02,ns_2008m03,ns_2008m04,ns_2008m05,ns_2008m06,ns_2008m07,ns_2008m08,ns_2008m09,ns_2008m10,ns_2008m11,ns_2008m12,ns_2009m01,ns_2009m02,ns_2009m03,ns_2009m04,ns_2009m05,ns_2009m06,ns_2009m07,ns_2009m08,ns_2009m09,ns_2009m10,ns_2009m11,ns_2009m12,ns_2010m01,ns_2010m02,ns_2010m03,ns_2010m04,ns_2010m05,ns_2010m06,ns_2010m07,ns_2010m08,ns_2010m09,ns_2010m10,ns_2010m11,ns_2010m12,ns_2011m01,ns_2011m02,ns_2011m03,ns_2011m04,ns_2011m05,ns_2011m06,ns_2011m07,ns_2011m08,ns_2011m09,ns_2011m10,ns_2011m11,ns_2011m12,ns_2012m01,ns_2012m02,ns_2012m03,ns_2012m04,ns_2012m05,ns_2012m06,ns_2012m07,ns_2012m08,ns_2012m09,ns_2012m10,ns_2012m11,ns_2012m12,ns_2013m01,ns_2013m02,ns_2013m03,ns_2013m04,ns_2013m05,ns_2013m06,ns_2013m07,ns_2013m08,ns_2013m09,ns_2013m10,ns_2013m11,ns_2013m12,ns_2014m01,ns_2014m02,ns_2014m03,ns_2014m04,ns_2014m05,ns_2014m06,ns_2014m07,ns_2014m08,ns_2014m09,ns_2014m10,ns_2014m11,ns_2014m12,ns_2015m01,ns_2015m02,ns_2015m03,ns_2015m04,ns_2015m05,ns_2015m06,ns_2015m07,ns_2015m08,ns_2015m09,ns_2015m10,ns_2015m11,ns_2015m12,ns_2016m01,ns_2016m02,ns_2016m03,ns_2016m04,ns_2016m05,ns_2016m06,ns_2016m07,ns_2016m08,ns_2016m09,ns_2016m10,ns_2016m11,ns_2016m12,ns_2017m01,ns_2017m02,ns_2017m03,ns_2017m04,ns_2017m05,ns_2017m06,ns_2017m07,ns_2017m08,ns_2017m09,ns_2017m10,ns_2017m11,ns_2017m12,ns_2018m01,ns_2018m02,ns_2018m03,ns_2018m04,ns_2018m05,ns_2018m06,ns_2018m07,ns_2018m08,ns_2018m09,ns_2018m10,ns_2018m11,ns_2018m12,ns_2019m01,ns_2019m02,ns_2019m03,ns_2019m04,ns_2019m05,ns_2019m06,ns_2019m07,ns_2019m08,ns_2019m09,ns_2019m10,ns_2019m11,ns_2019m12,ns_2020m01,ns_2020m02,ns_2020m03,ns_2020m04,ns_2020m05,ns_2020m06,ns_2020m07,ns_2020m08,ns_2020m09,ns_2020m10,ns_2020m11,ns_2020m12,ns_2021m01,ns_2021m02,ns_2021m03,ns_2021m04,ns_2021m05,ns_2021m06,ns_2021m07,ns_2021m08,ns_2021m09,ns_2021m10,ns_2021m11,ns_2021m12,ns_2022m01,ns_2022m02,ns_2022m03,ns_2022m04,ns_2022m05,ns_2022m06,ns_2022m07,ns_2022m08,ns_2022m09,ns_2022m10,ns_2022m11,ns_2022m12,ns_2023m01,ns_2023m02,ns_2023m03,ns_2023m04,ns_2023m05,ns_2023m06,ns_2023m07,ns_2023m08,ns_2023m09,ns_2023m10,ns_2023m11,ns_2023m12,ns_2024m01,ns_2024m02,ns_2024m03,ns_2024m04,ns_2024m05,ns_2024m06
0,1994_1,11.3,11.3,11.3,11.3,11.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

<div id="st-6-2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> <span>Clean-up vintages dataframe</span>
  </div>

In [466]:
# Call the appropriate function based on frequency and sector
if frequency == 'monthly':
    vintages_df = process_monthly(vintages)
elif frequency == 'quarterly':
    vintages_df = process_quarterly(vintages)
elif frequency == 'annual':
    vintages_df = process_annual(vintages)

In [467]:
vintages_df.head(30)

aux,vintages_date,ns_1997m01,ns_1997m02,ns_1997m03,ns_1997m04,ns_1997m05,ns_1997m06,ns_1997m07,ns_1997m08,ns_1997m09,ns_1997m10,ns_1997m11,ns_1997m12,ns_1998m01,ns_1998m02,ns_1998m03,ns_1998m04,ns_1998m05,ns_1998m06,ns_1998m07,ns_1998m08,ns_1998m09,ns_1998m10,ns_1998m11,ns_1998m12,ns_1999m01,ns_1999m02,ns_1999m03,ns_1999m04,ns_1999m05,ns_1999m06,ns_1999m07,ns_1999m08,ns_1999m09,ns_1999m10,ns_1999m11,ns_1999m12,ns_2000m01,ns_2000m02,ns_2000m03,ns_2000m04,ns_2000m05,ns_2000m06,ns_2000m07,ns_2000m08,ns_2000m09,ns_2000m10,ns_2000m11,ns_2000m12,ns_2001m01,ns_2001m02,ns_2001m03,ns_2001m04,ns_2001m05,ns_2001m06,ns_2001m07,ns_2001m08,ns_2001m09,ns_2001m10,ns_2001m11,ns_2001m12,ns_2002m01,ns_2002m02,ns_2002m03,ns_2002m04,ns_2002m05,ns_2002m06,ns_2002m07,ns_2002m08,ns_2002m09,ns_2002m10,ns_2002m11,ns_2002m12,ns_2003m01,ns_2003m02,ns_2003m03,ns_2003m04,ns_2003m05,ns_2003m06,ns_2003m07,ns_2003m08,ns_2003m09,ns_2003m10,ns_2003m11,ns_2003m12,ns_2004m01,ns_2004m02,ns_2004m03,ns_2004m04,ns_2004m05,ns_2004m06,ns_2004m07,ns_2004m08,ns_2004m09,ns_2004m10,ns_2004m11,ns_2004m12,ns_2005m01,ns_2005m02,ns_2005m03,ns_2005m04,ns_2005m05,ns_2005m06,ns_2005m07,ns_2005m08,ns_2005m09,ns_2005m10,ns_2005m11,ns_2005m12,ns_2006m01,ns_2006m02,ns_2006m03,ns_2006m04,ns_2006m05,ns_2006m06,ns_2006m07,ns_2006m08,ns_2006m09,ns_2006m10,ns_2006m11,ns_2006m12,ns_2007m01,ns_2007m02,ns_2007m03,ns_2007m04,ns_2007m05,ns_2007m06,ns_2007m07,ns_2007m08,ns_2007m09,ns_2007m10,ns_2007m11,ns_2007m12,ns_2008m01,ns_2008m02,ns_2008m03,ns_2008m04,ns_2008m05,ns_2008m06,ns_2008m07,ns_2008m08,ns_2008m09,ns_2008m10,ns_2008m11,ns_2008m12,ns_2009m01,ns_2009m02,ns_2009m03,ns_2009m04,ns_2009m05,ns_2009m06,ns_2009m07,ns_2009m08,ns_2009m09,ns_2009m10,ns_2009m11,ns_2009m12,ns_2010m01,ns_2010m02,ns_2010m03,ns_2010m04,ns_2010m05,ns_2010m06,ns_2010m07,ns_2010m08,ns_2010m09,ns_2010m10,ns_2010m11,ns_2010m12,ns_2011m01,ns_2011m02,ns_2011m03,ns_2011m04,ns_2011m05,ns_2011m06,ns_2011m07,ns_2011m08,ns_2011m09,ns_2011m10,ns_2011m11,ns_2011m12,ns_2012m01,ns_2012m02,ns_2012m03,ns_2012m04,ns_2012m05,ns_2012m06,ns_2012m07,ns_2012m08,ns_2012m09,ns_2012m10,ns_2012m11,ns_2012m12,ns_2013m01,ns_2013m02,ns_2013m03,ns_2013m04,ns_2013m05,ns_2013m06,ns_2013m07,ns_2013m08,ns_2013m09,ns_2013m10,ns_2013m11,ns_2013m12,ns_2014m01,ns_2014m02,ns_2014m03,ns_2014m04,ns_2014m05,ns_2014m06,ns_2014m07,ns_2014m08,ns_2014m09,ns_2014m10,ns_2014m11,ns_2014m12,ns_2015m01,ns_2015m02,ns_2015m03,ns_2015m04,ns_2015m05,ns_2015m06,ns_2015m07,ns_2015m08,ns_2015m09,ns_2015m10,ns_2015m11,ns_2015m12,ns_2016m01,ns_2016m02,ns_2016m03,ns_2016m04,ns_2016m05,ns_2016m06,ns_2016m07,ns_2016m08,ns_2016m09,ns_2016m10,ns_2016m11,ns_2016m12,ns_2017m01,ns_2017m02,ns_2017m03,ns_2017m04,ns_2017m05,ns_2017m06,ns_2017m07,ns_2017m08,ns_2017m09,ns_2017m10,ns_2017m11,ns_2017m12,ns_2018m01,ns_2018m02,ns_2018m03,ns_2018m04,ns_2018m05,ns_2018m06,ns_2018m07,ns_2018m08,ns_2018m09,ns_2018m10,ns_2018m11,ns_2018m12,ns_2019m01,ns_2019m02,ns_2019m03,ns_2019m04,ns_2019m05,ns_2019m06,ns_2019m07,ns_2019m08,ns_2019m09,ns_2019m10,ns_2019m11,ns_2019m12,ns_2020m01,ns_2020m02,ns_2020m03,ns_2020m04,ns_2020m05,ns_2020m06,ns_2020m07,ns_2020m08,ns_2020m09,ns_2020m10,ns_2020m11,ns_2020m12,ns_2021m01,ns_2021m02,ns_2021m03,ns_2021m04,ns_2021m05,ns_2021m06,ns_2021m07,ns_2021m08,ns_2021m09,ns_2021m10,ns_2021m11,ns_2021m12,ns_2022m01,ns_2022m02,ns_2022m03,ns_2022m04,ns_2022m05,ns_2022m06,ns_2022m07,ns_2022m08,ns_2022m09,ns_2022m10,ns_2022m11,ns_2022m12,ns_2023m01,ns_2023m02,ns_2023m03,ns_2023m04,ns_2023m05,ns_2023m06,ns_2023m07,ns_2023m08,ns_2023m09,ns_2023m10,ns_2023m11,ns_2023m12,ns_2024m01,ns_2024m02,ns_2024m03,ns_2024m04,ns_2024m05,ns_2024m06
0,1994-03-01,11.3,11.3,11.3,11.3,11.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

<div id="st-6-3">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> <span>Create releases</span>
  </div>

In [468]:
releases_df = create_releases(vintages_df, sector)
releases_df.head(30)

aux,vintages_date,manufacturing_release_1,manufacturing_release_2,manufacturing_release_3,manufacturing_release_4,manufacturing_release_5,manufacturing_release_6,manufacturing_release_7,manufacturing_release_8,manufacturing_release_9,manufacturing_release_10,manufacturing_release_11,manufacturing_release_12,manufacturing_release_13,manufacturing_release_14,manufacturing_release_15,manufacturing_release_16,manufacturing_release_17,manufacturing_release_18,manufacturing_release_19,manufacturing_release_20,manufacturing_release_21,manufacturing_release_22,manufacturing_release_23,manufacturing_release_24,manufacturing_release_25,manufacturing_release_26,manufacturing_release_27,manufacturing_release_28,manufacturing_release_29,manufacturing_release_30,manufacturing_release_31,manufacturing_release_32,manufacturing_release_33,manufacturing_release_34,manufacturing_release_35,manufacturing_release_36,manufacturing_release_37,manufacturing_release_38,manufacturing_release_39,manufacturing_release_40,manufacturing_most_recent
0,1994-03-01,11.3,11.3,11.3,11.3,11.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.3
1,1994-06-01,16.2,16.2,16.2,16.2,16.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.2
2,1994-09-01,16.3,16.3,16.3,16.3,16.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.3
3,1994-12-01,19.0,19.0,19.0,19.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0
4,1995-03-01,10.8,10.8,10.8,10.8,10.8,10.4,10.4,10.4,10.4,10.4,10.4,10.3,10.3,10.3,10.3,10.3,10.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.3
5,1995-06-01,5.9,5.9,5.9,5.9,5.9,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
6,1995-09-01,3.7,3.7,3.7,3.7,3.7,3.8,3.8,3.8,3.8,3.8,3.8,4.0,4.0,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
7,1995-12-01,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,-1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.5
8,1996-03-01,0.9,0.9,0.9,0.9,0.9,1.1,1.1,1.1,1.1,1.1,1.1,1.2,1.2,1.2,1.2,1.2,1.2,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9
9,1996-06-01,1.8,1.8,1.8,1.8,1.8,2.3,2.3,2.3,2.3,2.3,2.3,2.3,2.3,2.3,2.3,2.3,2.3,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.4


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-6" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

#### Check column type

In [469]:
print(releases_df[f'{sector}_release_1'].dtype)

float64


In [470]:
print(releases_df['vintages_date'].dtype)

datetime64[ns]


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="7">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: charter;">7.</span>
    <span style = "color: dark; font-family: charter;">
    Loading to SQL
    </span>
    </h2>

In [471]:
#horizon_df.to_sql(f'{sector}_{frequency}_growth_rates_horizon', engine, index=False, if_exists='replace')
#base_year_df.to_sql(f'{sector}_{frequency}_growth_rates_base_year', engine, index=False, if_exists='replace')
filtered_h_df.to_sql(f'{sector}_{frequency}_h', engine, index=False, if_exists='replace')
vintages_df.to_sql(f'{sector}_{frequency}_vintages', engine, index=False, if_exists='replace')
releases_df.to_sql(f'{sector}_{frequency}_releases', engine, index=False, if_exists='replace')

122

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-1-1" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-1-1" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select economic sector</b>.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>